In [3]:
import pandas as pd
import os

In [77]:
directory = "Data/2013-citibike"
rides_df = pd.DataFrame()

for filename in os.listdir(directory)[:2]:
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        # Read CSV file and append to the list
        df = pd.read_csv(file_path)
        print(file_path)
        rides_df = pd.concat([year_df, df]).drop(["start station id", "end station name",
                    "end station id", "bikeid","start station name"], axis=1)


Data/2013-citibike/201309-citibike-tripdata.csv


In [78]:
rides_df

,tripduration,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude,usertype,birth year,gender
0,1010,2013-09-01 00:00:02,2013-09-01 00:16:52,40.735324,-73.998004,40.715422,-74.011220,Subscriber,1974,1
1,1443,2013-09-01 00:00:09,2013-09-01 00:24:12,40.721816,-73.997203,40.737050,-73.990093,Customer,\N,0
2,1387,2013-09-01 00:00:16,2013-09-01 00:23:23,40.763406,-73.977225,40.739323,-74.008119,Subscriber,1992,1
3,405,2013-09-01 00:00:18,2013-09-01 00:07:03,40.751551,-73.993934,40.746745,-74.007756,Subscriber,1973,1
4,270,2013-09-01 00:00:20,2013-09-01 00:04:50,40.728419,-73.987140,40.722992,-73.979955,Subscriber,1984,1
...,...,...,...,...,...,...,...,...,...,...
1034354,93,2013-09-30 23:59:16,2013-10-01 00:00:49,40.707179,-74.008873,40.709056,-74.010434,Subscriber,1989,1
1034355,363,2013-09-30 23:59:21,2013-10-01 00:05:24,40.734161,-73.980243,40.746745,-74.007756,Customer,\N,0
1034356,562,2013-09-30 23:59:30,2013-10-01 00:08:52,40.730287,-73.990765,40.742065,-74.004432,Subscriber,1967,1
1034357,1237,2013-09-30 23:59:40,2013-10-01 00:20:17,40.760301,-73.998842,40.753202,-73.977987,Customer,\N,0


In [79]:
start_coord_df = rides_df[["start station latitude", "start station longitude"]]
start_coord_df.columns = ["lat", "lon"]
end_coord_df = rides_df[["end station latitude", "end station longitude"]]
end_coord_df.columns = ["lat", "lon"]

coord_df = pd.concat([coord_df, end_coord_df]).drop_duplicates().reset_index(drop=True)
coord_df

,lat,lon
0,40.720196,-73.989978
1,40.722055,-73.989111
2,40.729538,-73.984267
3,40.759291,-73.988597
4,40.726281,-73.989780
...,...,...
363,40.716633,-73.981933
364,40.744023,-73.976056
365,40.699773,-73.979927
366,40.755467,-73.986536


In [80]:
# !pip install geopandas
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Load the census tract shapefile
tracts = gpd.read_file('Data/tracts2020_shapefile/nyct2020.shp')
tracts = tracts.to_crs(epsg = 4326)

coords = [tuple(record) for record in coord_df.to_records(index=False)] #[(40.735324, -73.998004), (40.715595,	-73.987030)]

# Create a GeoDataFrame from the coordinates
geometry = [Point(lon, lat) for lat, lon in coords]
points_df = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")
points_df["lat"] = coord_df["lat"]
points_df["lon"] = coord_df["lon"]

# Perform a spatial join to match points to census tracts
ct_lookup = gpd.sjoin(points_df, tracts, how='left', predicate="within")[["lat", "lon", "CT2020"]]

# Display results
ct_lookup

,lat,lon,CT2020
0,40.720196,-73.989978,001800
1,40.722055,-73.989111,003601
2,40.729538,-73.984267,004001
3,40.759291,-73.988597,012101
4,40.726281,-73.989780,003800
...,...,...,...
363,40.716633,-73.981933,001200
364,40.744023,-73.976056,007001
365,40.699773,-73.979927,054300
366,40.755467,-73.986536,011300


In [82]:
# get start station CT from dictionary lookup
rides_df_ct = rides_df.merge(ct_lookup, left_on=["start station latitude", "start station longitude"], right_on=["lat", "lon"])
rides_df_ct.rename(columns={"CT2020":"start_CT"}, inplace=True)
rides_df_ct.drop(['lat','lon'], axis=1, inplace=True)

# get end station CT from dictionary lookup
rides_df_ct = rides_df_ct.merge(ct_lookup, left_on=["end station latitude", "end station longitude"], right_on=["lat", "lon"])
rides_df_ct.rename(columns={"CT2020":"end_CT"}, inplace=True)
rides_df_ct.drop(['lat','lon'], axis=1, inplace=True)
rides_df_ct

,tripduration,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude,usertype,birth year,gender,start_CT,end_CT
0,1010,2013-09-01 00:00:02,2013-09-01 00:16:52,40.735324,-73.998004,40.715422,-74.011220,Subscriber,1974,1,006300,002100
1,1443,2013-09-01 00:00:09,2013-09-01 00:24:12,40.721816,-73.997203,40.737050,-73.990093,Customer,\N,0,004100,005200
2,1387,2013-09-01 00:00:16,2013-09-01 00:23:23,40.763406,-73.977225,40.739323,-74.008119,Subscriber,1992,1,010400,007900
3,405,2013-09-01 00:00:18,2013-09-01 00:07:03,40.751551,-73.993934,40.746745,-74.007756,Subscriber,1973,1,010300,009901
4,270,2013-09-01 00:00:20,2013-09-01 00:04:50,40.728419,-73.987140,40.722992,-73.979955,Subscriber,1984,1,003800,002601
...,...,...,...,...,...,...,...,...,...,...,...,...
6649242,93,2013-09-30 23:59:16,2013-10-01 00:00:49,40.707179,-74.008873,40.709056,-74.010434,Subscriber,1989,1,000700,000700
6649243,363,2013-09-30 23:59:21,2013-10-01 00:05:24,40.734161,-73.980243,40.746745,-74.007756,Customer,\N,0,004400,009901
6649244,562,2013-09-30 23:59:30,2013-10-01 00:08:52,40.730287,-73.990765,40.742065,-74.004432,Subscriber,1967,1,005700,008300
6649245,1237,2013-09-30 23:59:40,2013-10-01 00:20:17,40.760301,-73.998842,40.753202,-73.977987,Customer,\N,0,011700,009400


In [ ]:
# derive features
